In [1]:
import pandas as pd
import numpy as np

finance_data = pd.read_csv('finance-data/finance-data.csv')

In [2]:
# Returns the intraday percentage change in stock price
def get_daily_percent_change(company):
    finance_data = pd.read_csv('finance-data/finance-data.csv').values
    company = finance_data[np.where(finance_data[:,7] == company)]
    dates = company[:,0]
    percent_change = (company[:,4]-company[:,1])/company[:,1]
    final = np.zeros((len(percent_change),2),dtype=object)
    final[:,0] = dates
    final[:,1] = percent_change
    return final

# Returns the interday percentage change in stock price
def get_overnight_percent_change(company):
    finance_data = pd.read_csv('finance-data/finance-data.csv').values
    company = finance_data[np.where(finance_data[:,7] == company)]
    dates = company[:,0]
    company = company[:,4]
    percent_change = np.zeros((company.shape[0],))
    percent_change[0] = 0
    for i in range(1,len(company)):
        percent_change[i] = (company[i]-company[i-1])/company[i-1]
    final = np.zeros((len(percent_change),2),dtype=object)
    final[:,0] = dates
    final[:,1] = percent_change
    return final

In [3]:
positive_emojis = eval(open('positive.emoji','r').readline())
negative_emojis = eval(open('negative.emoji','r').readline())

def convert_to_date(date):
    return date.split(' ')[0]

# Determines if an emoji is positively sentimented
def is_positive(emoji):
    for emo in positive_emojis:
        if emo in emoji:
            return True
    return False

# Determines if an emoji is negtively sentimented 
def is_negative(emoji):
    for emo in negative_emojis:
        if emo in emoji:
            return True
    return False

# Calculates the sentiment of a tweet
def calculate_sentiment(emojis):
    emojis = emojis.split(',')
    emojis = [emoji.encode() for emoji in emojis]
    sentiment = [0 for i in range(len(emojis))]
    for emoji in emojis:
        if is_positive(emoji):
            sentiment[emojis.index(emoji)] = 1
        elif is_negative(emoji):
            sentiment[emojis.index(emoji)] = -1
    return np.mean(np.array(sentiment))

# Converts the twitter data into sentiment scores for analysis
def get_twitter_data(company):
    twitter_data = pd.read_csv('twitter-data/{}.csv'.format(company))
    twitter_data['date'] = twitter_data['date'].apply(convert_to_date)
    twitter_data['emojis'] = twitter_data['emojis'].apply(calculate_sentiment)
    final = np.zeros((len(twitter_data),2),dtype=object)
    final[:,0] = twitter_data['date'].values
    final[:,1] = twitter_data['emojis'].values
    return final


In [4]:
# Extracts the daily sentiment score
def filter_twitter_data(twitter_data, finance_data):
    daily_sentiment = np.zeros_like(finance_data,dtype=object)
    for i in range(len(finance_data)):
        date = finance_data[i,0]
        sentiment = twitter_data[np.where(twitter_data[:,0] == date)]
        average_sentiment = np.mean(sentiment[:,1])
        daily_sentiment[i,0] = date
        daily_sentiment[i,1] = average_sentiment
    return daily_sentiment

# Calculates the percentage change in sentiment
def score_percent_change(scores):
    final = np.zeros_like(scores)
    final[0,0], final[0,1] = scores[0,0], 0
    for i in range(1,len(scores)):
        final[i,0] = scores[i,0]
        try:
            final[i,1] = (scores[i,1]-scores[i-1,1])/scores[i-1,1]
        except:
            final[i,1] = 0
    return final



In [5]:
import pandas as pd
import numpy as np

# Prepares the data
data = pd.read_csv('finance-data/finance-data.csv')
data = data['ticker'].values
companies = list(np.unique(data))
companies.remove('CAT')

In [6]:
import csv

# Final preparations for the granger test to be run in a R kernel
for company in companies:
    twitter_data = get_twitter_data(company)
    finance_data = get_daily_percent_change(company)

    scores = filter_twitter_data(twitter_data, finance_data)
    scores = scores[:len(scores)-1]
    scores = score_percent_change(scores)
    finance_data = finance_data[:len(finance_data)-1]

    fp = open('analysis/{}.csv'.format(company),'w')
    writer = csv.writer(fp)
    for i in range(len(scores)):
        row = [finance_data[i,0], finance_data[i,1], scores[i, 1]]
        writer.writerow(row)
    fp.close()

/home/pstance/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/pstance/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/pstance/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars
